In [1]:
from scipy.stats import norm
import scipy.integrate as ing
import numpy as np

In [2]:
def calculate_performance(M, n, mu, sigma, tau):
    qstar = norm.ppf(tau, mu, sigma)
    profit_qstar = calculate_profit(qstar, tau)
    profit_np_values = []
    profit_p_values = []

    for _ in range(M):
        data = norm.rvs(size=n, loc=mu, scale=sigma)

        # parametric
        sample_mean = np.mean(data)
        sample_variance_unadj = np.var(data)
        qparam = norm.ppf(tau, sample_mean, np.sqrt(sample_variance_unadj))
        profit_param = calculate_profit(qparam, tau)  # compute parametric profit
        profit_p_values.append(profit_qstar - profit_param)

        # non parametric
        qnonparam = get_order_statistic(data, int(np.ceil(tau * n)) - 1)  # to improve
        profit_np = calculate_profit(qnonparam, tau)
        profit_np_values.append(profit_qstar - profit_np)  #compute np profit

    plr_np = (np.sum(abs(profit_np_values / profit_qstar)) / M)
    plr_p = (np.sum(abs(profit_p_values / profit_qstar)) / M)
    plr_ratio = plr_np / plr_p
    return plr_ratio


def get_order_statistic(data, order):
    return np.partition(np.asarray(data), order)[order]


def int_normal(y):
    f = norm.pdf(y, loc=120, scale=15)
    return y * f


def calculate_profit(Q, tau):
    c = 1 - tau
    p = 1
    t1 = p * ing.quad(int_normal, 0, Q)[0]
    t2 = p * Q * norm.sf(Q, loc=120, scale=15)  # .sf is the survival function, i.e., 1-F(x)
    t3 = c * Q

    profit = t1 + t2 - t3
    return profit

In [3]:
def main():
    n = [10, 50, 100, 200]
    tau = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    M = 1000
    mu = 120
    sigma = 15

    for i in n:
        for j in tau:
            print(calculate_performance(M, i, mu, sigma, j))


main()

4.164121448809176
1.350154366949227
1.350981276052725
1.5578872147085883
1.6288526147174818
1.620709304294041
1.6485265000559648
1.3535937843398052
3.946436933155042
3.057704960259069
1.7273595069492753
1.629524895504108
1.5510206573583079
1.6219097458598788
1.592615668870078
1.6021786310281831
1.8015400891084878
3.004409964798946
3.779483988688139
1.886454856523351
1.5485840342181327
1.5383931808693072
1.6267797404654263
1.5076950998145493
1.5744163443913282
1.95058353894599
4.564804221915
3.8770642107408606
1.9299500348024312
1.676077559160322
1.510959961798838
1.4459938136193948
1.5229611399421548
1.7223625824591682
1.895440351628185
4.237907757627279
